# **Minimax with Alpha-Beta Pruning – Enhanced Jupyter Notebook (2025 Edition)**

This notebook builds on **plain Minimax** and introduces **Alpha-Beta Pruning** with:
- Full theory + intuition
- Step-by-step function walkthrough
- Instrumented code (which nodes are pruned)
- Node-count comparison (Minimax vs Alpha-Beta)
- Visualization with Tic-Tac-Toe game


## **1. Why Alpha-Beta Pruning?**

Minimax explores **all nodes** in the game tree.

For branching factor `b` and depth `d`:
- Time = `O(b^d)` → grows extremely fast.

**Alpha-Beta Pruning** reduces the number of nodes we evaluate **without changing the final result**.

### Idea:

We keep two values during recursion:
- **α (alpha)** → best value that MAX can guarantee **so far** (lower bound)
- **β (beta)** → best value that MIN can guarantee **so far** (upper bound)

Whenever we find a situation where:
```text
alpha ≥ beta
```
we can **stop exploring further children** of that node → they will **never affect** the final decision.


## **2. Reusing Tic-Tac-Toe Helpers**

We reuse the same board helpers as in the Minimax notebook.

If you are running this independently, execute the cell below to re-define them.


In [1]:
from typing import List, Optional, Tuple

Board = List[List[str]]

def create_empty_board() -> Board:
    return [[' ' for _ in range(3)] for _ in range(3)]

def print_board(board: Board) -> None:
    for i, row in enumerate(board):
        print(' | '.join(row))
        if i < 2:
            print('-' * 5)
    print()

def check_winner(board: Board) -> Optional[str]:
    lines = []

    for i in range(3):
        lines.append(board[i])
        lines.append([board[0][i], board[1][i], board[2][i]])

    lines.append([board[0][0], board[1][1], board[2][2]])
    lines.append([board[0][2], board[1][1], board[2][0]])

    for line in lines:
        if line[0] != ' ' and line.count(line[0]) == 3:
            return line[0]
    return None

def is_full(board: Board) -> bool:
    return all(board[i][j] != ' ' for i in range(3) for j in range(3))

def is_terminal(board: Board) -> bool:
    return check_winner(board) is not None or is_full(board)

def get_empty_positions(board: Board):
    cells = []
    for i in range(3):
        for j in range(3):
            if board[i][j] == ' ':
                cells.append((i,j))
    return cells

def evaluate(board: Board) -> int:
    winner = check_winner(board)
    if winner == 'X':
        return 1
    elif winner == 'O':
        return -1
    else:
        return 0


## **3. Minimax with Alpha-Beta – Implementation**

We now implement:

```text
alphabeta(board, depth, alpha, beta, is_max_turn)
```

- `alpha` starts from `-∞`
- `beta` starts from `+∞`

We also **count nodes** to compare with plain minimax.


In [2]:
node_count_ab = 0
pruned_nodes = 0

def alphabeta(board: Board, depth: int, alpha: float, beta: float, is_max_turn: bool):
    global node_count_ab, pruned_nodes
    node_count_ab += 1

    if is_terminal(board):
        return evaluate(board), None

    if is_max_turn:
        best_score = float('-inf')
        best_move = None
        for (i,j) in get_empty_positions(board):
            board[i][j] = 'X'
            score, _ = alphabeta(board, depth+1, alpha, beta, False)
            board[i][j] = ' '
            if score > best_score:
                best_score = score
                best_move = (i,j)
            alpha = max(alpha, best_score)
            if beta <= alpha:
                pruned_nodes += 1
                break
        return best_score, best_move
    else:
        best_score = float('inf')
        best_move = None
        for (i,j) in get_empty_positions(board):
            board[i][j] = 'O'
            score, _ = alphabeta(board, depth+1, alpha, beta, True)
            board[i][j] = ' '
            if score < best_score:
                best_score = score
                best_move = (i,j)
            beta = min(beta, best_score)
            if beta <= alpha:
                pruned_nodes += 1
                break
        return best_score, best_move

# Test from initial position
board = create_empty_board()
node_count_ab = 0
pruned_nodes = 0
score, move = alphabeta(board, 0, float('-inf'), float('inf'), True)
print("Best move:", move, "score:", score)
print("Nodes visited (Alpha-Beta):", node_count_ab)
print("Approx. pruned branches:", pruned_nodes)


Best move: (0, 0) score: 0
Nodes visited (Alpha-Beta): 18297
Approx. pruned branches: 8180


## **4. Comparing Minimax vs Alpha-Beta (Node Count)**

We now implement a plain minimax again **inside this notebook** for comparison.


In [3]:
node_count_mm = 0

def minimax_plain(board: Board, is_max_turn: bool):
    global node_count_mm
    node_count_mm += 1

    if is_terminal(board):
        return evaluate(board), None

    if is_max_turn:
        best_score = float('-inf')
        best_move = None
        for (i,j) in get_empty_positions(board):
            board[i][j] = 'X'
            score, _ = minimax_plain(board, False)
            board[i][j] = ' '
            if score > best_score:
                best_score = score
                best_move = (i,j)
        return best_score, best_move
    else:
        best_score = float('inf')
        best_move = None
        for (i,j) in get_empty_positions(board):
            board[i][j] = 'O'
            score, _ = minimax_plain(board, True)
            board[i][j] = ' '
            if score < best_score:
                best_score = score
                best_move = (i,j)
        return best_score, best_move

board = create_empty_board()
node_count_mm = 0
score_mm, move_mm = minimax_plain(board, True)

board = create_empty_board()
node_count_ab = 0
pruned_nodes = 0
score_ab, move_ab = alphabeta(board, 0, float('-inf'), float('inf'), True)

print("Plain Minimax -> move:", move_mm, "score:", score_mm, "nodes:", node_count_mm)
print("Alpha-Beta    -> move:", move_ab, "score:", score_ab, "nodes:", node_count_ab)


Plain Minimax -> move: (0, 0) score: 0 nodes: 549946
Alpha-Beta    -> move: (0, 0) score: 0 nodes: 18297


You can observe that:
- **Both algorithms choose the same move and score** (correctness preserved)
- **Alpha-Beta visits fewer nodes** (performance gain)


## **5. Visual Animation: AI with Alpha-Beta**

We now build a Tic-Tac-Toe AI that uses **Alpha-Beta** and animate its moves.


In [6]:
from IPython.display import clear_output
import time

def alphabeta_best_move(board: Board, is_max_turn: bool):
    score, move = alphabeta(board, 0, float('-inf'), float('inf'), is_max_turn)
    return move

def play_ai_vs_ai_ab(delay: float = 0.7):
    board = create_empty_board()
    is_max_turn = True

    while not is_terminal(board):
        clear_output(wait=True)
        print("Current player (Alpha-Beta):", 'X' if is_max_turn else 'O')
        print_board(board)
        move = alphabeta_best_move(board, is_max_turn)
        if move is None:
            break
        i, j = move
        board[i][j] = 'X' if is_max_turn else 'O'
        is_max_turn = not is_max_turn
        time.sleep(delay)

    clear_output(wait=True)
    print("Final Board (Alpha-Beta Game):")
    print_board(board)
    winner = check_winner(board)
    if winner:
        print("Winner:", winner)
    else:
        print("Result: Draw")


# Run this in Jupyter to see the game animation:
play_ai_vs_ai_ab()


Final Board (Alpha-Beta Game):
X | X | O
-----
O | O | X
-----
X | O | X

Result: Draw


## **6. Time Complexity with Alpha-Beta**

Worst-case complexity remains `O(b^d)`.

However, **with good move ordering**, Alpha-Beta can approach:
```text
O(b^(d/2))
```
This is a **huge speedup**, effectively allowing us to **search twice as deep** in the same time.

### Key Trick: Move Ordering
If we explore **promising moves first**, pruning becomes more effective.


## **7. Practice Tasks (Alpha-Beta)**

1. Modify Alpha-Beta to **print which moves are pruned** at each depth.

2. Add a **depth limit** and use heuristic evaluation when depth is reached.

3. Apply Alpha-Beta to a **Connect-4** or **3×3×3 Tic-Tac-Toe** game.

4. Experiment with different **move ordering strategies** and compare node counts.

5. Add a human vs AI mode using Alpha-Beta (take input from user and alternate).

